# Test Audio Sender

This file is used for sending audio data to the device. It measures the accuracy based on individual audio files.

**Note that the software on the MCU must match this!**

In [169]:
AUDIO_FOLDER_PATH = "./dataset_mix"
POSITIVE_PATH = f"{AUDIO_FOLDER_PATH}/positive"
NEGATIVE_PATH = f"{AUDIO_FOLDER_PATH}/negative"

In [170]:
import numpy as np
import pandas as pd
import gc
import os
import librosa
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import serial
import struct

In [171]:
# Collecte positive audio files and their classes
positive_audio_files = []
positive_audio_file_classes = []
# Look each file in the audio folder
audio_class_folders = os.listdir(POSITIVE_PATH)
# Loop each class folder
for audio_class_folder in audio_class_folders:
    # Assemble full audio class folder path.
    audio_class_folder_path = os.path.join(POSITIVE_PATH, audio_class_folder)
    print("Processing class folder: ", audio_class_folder_path)
    # Get all files in the audio class folder
    audio_class_files = os.listdir(audio_class_folder_path)
    # Loop each audio file in the audio class folder
    for audio_class_file in audio_class_files:
        # Assemble full audio file path.
        audio_file_path = os.path.join(audio_class_folder_path, audio_class_file)
        # Append the audio file path to the positive_audio_files list
        positive_audio_files.append(audio_file_path)
        # Append the audio class to the audio_file_classes list
        positive_audio_file_classes.append(audio_class_folder)
print("positive_audio_files length: ", len(positive_audio_files))
print("positive_audio_file_classes length: ", len(positive_audio_file_classes))

Processing class folder:  ./dataset_mix/positive/Car
Processing class folder:  ./dataset_mix/positive/Comm
Processing class folder:  ./dataset_mix/positive/Motorcycle
positive_audio_files length:  18597
positive_audio_file_classes length:  18597


In [172]:
positive_audio_file_classes[0]

'Car'

In [173]:
# Collecte negative audio files and their classes
negative_audio_files = []
negative_audio_file_classes = []
# Look each file in the audio folder
audio_class_folders = os.listdir(NEGATIVE_PATH)
# Loop each class folder
for audio_class_folder in audio_class_folders:
    # Assemble full audio class folder path.
    audio_class_folder_path = os.path.join(NEGATIVE_PATH, audio_class_folder)
    print("Processing class folder: ", audio_class_folder_path)
    # Get all files in the audio class folder
    audio_class_files = os.listdir(audio_class_folder_path)
    # Loop each audio file in the audio class folder
    for audio_class_file in audio_class_files:
        # Assemble full audio file path.
        audio_file_path = os.path.join(audio_class_folder_path, audio_class_file)
        # Append the audio file path to the negative_audio_files list
        negative_audio_files.append(audio_file_path)
        # Append the audio class to the audio_file_classes list
        negative_audio_file_classes.append(audio_class_folder)
print("negative_audio_files length: ", len(negative_audio_files[0:10]))
print("negative_audio_file_classes length: ", len(negative_audio_file_classes[0:10]))

Processing class folder:  ./dataset_mix/negative/background
negative_audio_files length:  10
negative_audio_file_classes length:  10


In [174]:
# Shuffle the audio files and classes with the same seed.
seed = 42
np.random.seed(seed)
np.random.shuffle(positive_audio_files)

np.random.seed(seed)
np.random.shuffle(positive_audio_file_classes)

np.random.seed(seed)
np.random.shuffle(negative_audio_files)

In [175]:
# Hot end code the labels.
label_encoder = LabelEncoder()
positive_audio_file_classes_categorical = to_categorical(label_encoder.fit_transform(positive_audio_file_classes))
print("Example of audio_file_classes_categorial: ", positive_audio_file_classes_categorical[0:10])

Example of audio_file_classes_categorial:  [[0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [230]:
# Connect to MCU via serial.
print("Configuring serial port...")
ser = serial.Serial(
    port='/dev/ttyACM0',  # Change this to your actual port, e.g., 'COM3' on Windows, '/dev/ttyS0' on Linux
    baudrate=921600,       # Set baud rate to 921600
    bytesize=serial.EIGHTBITS,
    parity=serial.PARITY_NONE,
    stopbits=serial.STOPBITS_ONE,
    timeout=0.1,           # Set timeout for reading
    write_timeout=None     # Wait indefinitely until all data is sent
)

Configuring serial port...


In [231]:
if ser.is_open:
    print(f"Serial port {ser.port} opened at {ser.baudrate} baud.")

Serial port /dev/ttyACM0 opened at 921600 baud.


In [232]:
def calc_accuracy(y_true, y_pred):
    success = 0
    for i in range(len(y_pred)):
        if y_pred[i] == y_true[i]:
            success += 1
    res = round(success / len(y_pred), 3)
    return res

In [233]:
audioSent = 0
audioFilesProcessed = 0
results = []
realResults = []

In [234]:
positive_audio_file_classes_categorical[0]

array([0., 1., 0.])

In [235]:
import time
dataSent = 0

def streamAudioFile(file_path, realClass, i):
    global audioSent
    global results
    global realResults
    global audioBuffer
    global dataSent
    voted = False
    # Append the real result to the results list.
    realResults.append(np.argmax(realClass))
    #print(f"Adding realResults: {realResults}")
    # Check file format.
    if not file_path.endswith('.wav'):
        print("Error: Only .wav files are supported.")
        return
    # Load the audio file.
    audio_data, sr = librosa.load(file_path, sr=None)
    # Convert sample rate to 16kHz.
    if sr != 16000:
        audio_data = librosa.resample(y=audio_data, orig_sr=sr, target_sr=16000)
        sr = 16000
    # Convert the audio data to a numpy array.
    audio_data = np.array(audio_data, dtype=np.float32)
    # Send the audio data to the MCU.
    #j = 0
    #while j+4000 < len(audio_data):
    #for j in range(len(audio_data)):
        #dataSent = (dataSent + 1) % 4000
        #start_time = time.time()
        #data_to_send = audio_data[j:j+4000].tobytes()
        #j += 4000
    start_time = time.time()
    ser.write(audio_data.tobytes())
    while ser.in_waiting == 0:
        time.sleep(0.01)
    end_time = time.time()
    print(f"Time to send file: {end_time - start_time} seconds")
    # Wait til MCU returns "s:ok"
    #while ser.in_waiting == 0:
    #    time.sleep(0.01)  # Sleep for 10 milliseconds
    #if dataSent == 0 :
    while ser.in_waiting > 0:
        response = ser.readline().decode('utf-8').strip()
        if response.startswith("e:"):
            print(response)
            return
        if response.startswith("c:"):
            print(response)
        if response.startswith("s:"):
            print(response)
        if response.startswith("v:"):
            voted = True
            #print(response)
            #print(f"Adding {int(response[2:])} to results")
            response = int(response[2:])
            results.append(response)
            if len(results) > len(realResults):
                realResults.append(9)
            print(f"Accuracy: {calc_accuracy(realResults, results)}")
        #end_time = time.time()
        #print(f"Time to send file: {end_time - start_time} seconds")
    # If result is shorter than realResults, append 9 to results.
    if i > 0 and len(results) < len(realResults) and not voted:
        results.append(9)
        print(f"Accuracy: {calc_accuracy(realResults, results)}")

In [236]:

total_positive_audio_files = len(positive_audio_files)

# Loop each audio file.
negative_audio_file_pointer = 0
for i in range(total_positive_audio_files):
    # Get audio file path.
    audio_file = positive_audio_files[i]
    # Print audio file path and class.
    print("Processing audio file: ", audio_file)
    print("Processing audio file class: ", positive_audio_file_classes[i])
    # Stream positive audio file to MCU.
    streamAudioFile(audio_file, positive_audio_file_classes_categorical[i], i)
    # Print nagtive audio file path and class.
    print("Processing negative audio file: ", negative_audio_files[negative_audio_file_pointer])
    # Stream negative audio file to MCU.
    streamAudioFile(negative_audio_files[negative_audio_file_pointer], [0,0,0,1], i)
    # Increment the negative audio file pointer.
    negative_audio_file_pointer = (negative_audio_file_pointer + 1) % len(negative_audio_files)

Processing audio file:  ./dataset_mix/positive/Comm/Y99CehZOeqlQ_30.000_40.000.wav
Processing audio file class:  Comm
Time to send file: 27.955339431762695 seconds
s:Inference time: 658.71 ms
s:ok
e:Classification failed
Processing negative audio file:  ./dataset_mix/negative/background/2019-11-13-15-55_Schleusinger-Allee_70Kmh_124454655_ME_CH12-BG.wav
Time to send file: 5.726912975311279 seconds
s:Inference time: 658.84 ms
s:ok
e:Classification failed
Processing audio file:  ./dataset_mix/positive/Comm/Y7Rw3S1JRfbc_30.000_40.000.wav
Processing audio file class:  Comm
Time to send file: 29.34826636314392 seconds
s:Inference time: 658.64 ms
s:ok
e:Classification failed
Processing negative audio file:  ./dataset_mix/negative/background/2019-10-22-08-40_Fraunhofer-IDMT_30Kmh_9885440_ME_CH12-BG.wav
Time to send file: 5.9019834995269775 seconds
s:Inference time: 658.89 ms
s:ok
e:Classification failed
Processing audio file:  ./dataset_mix/positive/Car/2019-10-22-08-40_Fraunhofer-IDMT_30Kmh_2

KeyboardInterrupt: 

In [237]:
# Close the serial port
ser.close()